In [1]:
import datetime
import numpy as np
import pandas as pd

from tqdm import tqdm, trange

from airsim.collections import AirObject, AirEnv, RadarSystem, ControlPoint, Supervisor
from airsim.time import Time

In [2]:
def save_cp_data(cp_data, cp_data_dir, data_usage_aim):
    timestamp = datetime.datetime.now().strftime('%d%m%H%M%S')
    filename = f'{cp_data_dir}/{data_usage_aim}/data_{timestamp}.csv'
    cp_data.to_csv(filename)
    return filename

In [3]:
def generate_linear_tracks(n_tracks, axis='xyz'):
    tracks = []
    
    Rx_0 = np.random.uniform(-1.0e+6, 1.0e+6, size=n_tracks)
    Ry_0 = np.random.uniform(-1.0e+6, 1.0e+6, size=n_tracks)
    Rz_0 = np.random.uniform(1.0e+4 - 1.0e+2, 1.0e+4 + 1.0e+2, size=n_tracks)

    Vx = np.random.uniform(-0.4, 0.4, size=n_tracks) if 'x' in axis else [0.0] * n_tracks
    Vy = np.random.uniform(-0.4, 0.4, size=n_tracks) if 'y' in axis else [0.0] * n_tracks
    Vz = np.random.uniform(-0.01, 0.01, size=n_tracks) if 'z' in axis else [0.0] * n_tracks

    for i in range(n_tracks):
        Rx_0_i = Rx_0[i]
        Ry_0_i = Ry_0[i]
        Rz_0_i = Rz_0[i]
        Vx_i = Vx[i]
        Vy_i = Vy[i]
        Vz_i = Vz[i]
        
        tracks.append(
            lambda t, Rx_0_i=Rx_0_i, Ry_0_i=Ry_0_i, Rz_0_i=Rz_0_i, Vx_i=Vx_i, Vy_i=Vy_i, Vz_i=Vz_i: np.array([Rx_0_i + Vx_i * t, Ry_0_i + Vy_i * t, Rz_0_i + Vz_i * t])
        )

    return tracks

In [4]:
def generate_cp_data(t_max_seconds, axis='xyz'):
    t_min = 0
    t_max = t_max_seconds * 1000
    dt = 50

    supervisor = Supervisor(
        air_env=AirEnv(air_objects=[
            AirObject(track=ao_track) for ao_track in generate_linear_tracks(2, axis=axis)
        ]),
        radar_systems=[
            RadarSystem(position=np.array([0, 0, 0]),
                        detection_radius=1e+308,
                        error=1.0,
                        detection_fault_probability=0.01,
                        detection_period=250,
                        detection_delay=np.random.randint(0, 250//dt) * dt),
            RadarSystem(position=np.array([0, 0, 0]),
                        detection_radius=1e+308,
                        error=1.0,
                        detection_fault_probability=0.01,
                        detection_period=250,
                        detection_delay=np.random.randint(0, 250//dt) * dt),
        ]
    )

    supervisor.run(t_min, t_max, dt)

    return supervisor.get_data()

## Данные для обучения

In [5]:
cp_data_dir = 'CP_data' 
data_usage_aim = 'train'

In [6]:
n_generations = 60

for i in range(n_generations):
    cp_data = generate_cp_data(t_max_seconds=1800, axis='x')
    filename = save_cp_data(cp_data, cp_data_dir, data_usage_aim)

0.37260973794388874
-0.3178962417969315


Running system: 100%|██████████████████████████████████████████████| 21/21 [00:00<00:00, 546.39it/s]


0.28237881263483267
0.06654907327700094


Running system: 100%|██████████████████████████████████████████████| 21/21 [00:00<00:00, 694.28it/s]


-0.18998458364525492
0.12726477775814127


...

0.26980614097730626
-0.2634776474653829


Running system: 100%|██████████████████████████████████████████████| 21/21 [00:00<00:00, 736.19it/s]


-0.20634254634783344
0.011146210448250593


Running system: 100%|██████████████████████████████████████████████| 21/21 [00:00<00:00, 663.72it/s]


KeyboardInterrupt: 

## Данные для тестирования

In [7]:
cp_data_dir = 'CP_data' 
data_usage_aim = 'test'

In [8]:
n_generations = 20

for i in range(n_generations):
    cp_data = generate_cp_data(t_max_seconds=300, axis='x')
    filename = save_cp_data(cp_data, cp_data_dir, data_usage_aim)

Running system: 100%|██████████████████████████████████████████| 6001/6001 [00:07<00:00, 776.38it/s]
